# Heroes of Pymoli Data Analysis

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
csv_file = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(csv_file)
raw = pd.read_csv(csv_file)

In [2]:
# Head for reference
raw.head()
#raw.columns

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

In [3]:
# Counting the number of unique SN assumed to be ID names
player_count = pd.DataFrame(data = {'Total Number of Players' : raw['SN'].nunique()}, index = [0])
#player_count.head()

## Purchasing Analysis

In [4]:
# Counting the unique number of item IDs
purchase_analysis = pd.DataFrame(data = {'Number of Unique Items' : raw['Item ID'].nunique()}, index = [0])

# Taking the mean of the item purchase price
purchase_analysis['Average Purchase Price'] = raw['Price'].mean()

#Changing the format to be currency
purchase_analysis['Average Purchase Price'] = purchase_analysis['Average Purchase Price'].map('${:.2f}'.format)

# Calculating the number of unique purchases
purchase_analysis['Total Number of Purchases'] = raw['Purchase ID'].nunique()

#Calculating the sum of all of the purchases
purchase_analysis['Total Revenue'] = raw['Price'].sum()

# Changing the price to be 
purchase_analysis['Total Revenue'] = purchase_analysis['Total Revenue'].map('${:,.2f}'.format)
#purchase_analysis.head()

## Gender Demographics

In [5]:
# Generating a data frame with the number of players group by gender
group_gender = raw.groupby('Gender')
gender_demographics = pd.DataFrame(group_gender.nunique()['SN'])
gender_demographics = gender_demographics.rename(columns = {'SN' : 'Total Count'})

# Calculating the percentage of players by gender
gender_demographics['Percentage of Players'] = (gender_demographics['Total Count']
                                                .div(gender_demographics['Total Count'].sum()) * 100)
gender_demographics['Percentage of Players'] = gender_demographics['Percentage of Players'].map('{:.2f}%'.format)

# Sorting the number of players in descending order
gender_demographics = gender_demographics.sort_values(["Total Count"], ascending = False)
#gender_demographics.head()

## Purchasing Analysis (Gender)

In [6]:
# Generating a data frame with the number of purchases group by gender
purchasing_analysis = pd.DataFrame(group_gender.count()['Item ID'])
purchasing_analysis = purchasing_analysis.rename(columns = {'Item ID' : 'Purchase Count'})

# Calculating the average purchase price by gender
purchasing_analysis['Average Purchase Price'] = group_gender.mean()['Price']

# Calculating the Average value of the purchases by gender
purchasing_analysis['Total Purchase Value'] = group_gender.sum()['Price']

# Calculating the average total purchase per person by gender
purchasing_analysis['Avg Total Purchase per Person'] = (purchasing_analysis['Total Purchase Value'] /
                                                        gender_demographics['Total Count'])

#Convert to currency format
purchasing_analysis['Total Purchase Value'] = purchasing_analysis['Total Purchase Value'].map('${:,.2f}'.format)
purchasing_analysis['Average Purchase Price'] = purchasing_analysis['Average Purchase Price'].map('${:,.2f}'.format)
purchasing_analysis['Avg Total Purchase per Person'] = (purchasing_analysis['Avg Total Purchase per Person'].
                                                        map('${:,.2f}'.format))

#purchasing_analysis.head()

## Age Demographics

In [7]:
# Creating bins for age groups
bin_age = [0, 10, 15, 20, 25, 30, 35, 40, 200]
bin_label = ['<10', '10-14', '15 - 19', '20-24', '25-29', '30-34', '35-39', '40+']

# Adding the age groups to the original data frame
raw['Age Group'] = pd.cut(raw['Age'], bin_age, labels = bin_label, right = False)

# Creating a data frame group by the age groups
group_age = raw.groupby('Age Group')

# Generating a new data frame counting the number of unique players
age_demographics = pd.DataFrame(group_age.nunique()['SN'])

# Renaming the columns to Total Count
age_demographics = age_demographics.rename(columns = {'SN' : 'Total Count'})

# Calculating the percentage of players in each age group
age_demographics['Percentage of Players'] = (age_demographics['Total Count'] / age_demographics['Total Count'].sum() * 100)

# Formatting the Percentage of Players to an actual percentage
age_demographics['Percentage of Players'] = age_demographics['Percentage of Players'].map('{:.2f}%'.format)

#age_demographics.head(8)

## Purchasing Analysis (Age)

In [8]:
# Generating initial data frame with purchase count
purchasing_analysis_age = pd.DataFrame(group_age.count()['Item ID'])
purchasing_analysis_age = purchasing_analysis_age.rename(columns = {'Item ID' : 'Purchase Count'})

# Calculating the Average Purchase Price
purchasing_analysis_age['Average Purchase Price'] = group_age.mean()['Price']

# Calculating total purchase value
purchasing_analysis_age['Total Purchase Value'] = group_age.sum()['Price']

# Calculating average total purchase per person
purchasing_analysis_age['Avg Total Purchase per Person'] = (purchasing_analysis_age['Total Purchase Value'] /
                                                            age_demographics['Total Count'])

# Formatting to currency
purchasing_analysis_age['Average Purchase Price'] = purchasing_analysis_age['Average Purchase Price'].map('${:,.2f}'.format)
purchasing_analysis_age['Total Purchase Value'] = purchasing_analysis_age['Total Purchase Value'].map('${:,.2f}'.format)
purchasing_analysis_age['Avg Total Purchase per Person'] = (purchasing_analysis_age['Avg Total Purchase per Person']
                                                            .map('${:,.2f}'.format))

purchasing_analysis_age.head(8)

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15 - 19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spendors

In [9]:
# Generating a df group by SN with the Purchase Count
group_sn = raw.groupby('SN')
top_spendors = pd.DataFrame(group_sn.count()['Purchase ID'])
top_spendors = top_spendors.rename(columns = {'Purchase ID' : 'Purchase Count'})

# Calculating the Average Purchase Price
top_spendors['Average Purchase Price'] = group_sn.mean()['Price']

# Calculating the Total Purchase Value
top_spendors['Total Purchase Value'] = group_sn.sum()['Price']

# Sorting the table by the Total Purchase Value
top_spendors = top_spendors.sort_values('Total Purchase Value', ascending = False)

# Formatting to currency
top_spendors['Average Purchase Price'] = top_spendors['Average Purchase Price'].map('${:,.2f}'.format)
top_spendors['Total Purchase Value'] = top_spendors['Total Purchase Value'].map('${:,.2f}'.format)

#top_spendors.head()

In [10]:
# Create new DF with the Item Id and name
group_item = raw.groupby(['Item ID', 'Item Name'])
popular_item = pd.DataFrame(group_item.nunique()['Purchase ID'])

# Rename to Purchase Count
popular_item = popular_item.rename(columns = {'Purchase ID' : 'Purchase Count'})

# Add Item Price columns
popular_item['Item Price'] = group_item

popular_item.head()

,,Purchase Count,Item Price
Item ID,Item Name,,
0,Splinter,4,"((0, Splinter), [Purchase ID, SN, Age, Gender])"
1,Crucifer,3,"((1, Crucifer), [Purchase ID, SN, Age])"
2,Verdict,6,"((2, Verdict), [Purchase ID, SN, Age, Gender, ..."
3,Phantomlight,6,"((3, Phantomlight), [Purchase ID, SN, Age, Gen..."
4,Bloodlord's Fetish,5,"((4, Bloodlord's Fetish), [Purchase ID, SN, Ag..."
